In [1]:
import re
from owlready2 import *
from pybliometrics.scopus import ScopusSearch
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
#load the ontology and perform reasoning
onto_new= "afo_upd1"

new_world = owlready2.World()
onto = get_ontology("./ontologies/{}.owl".format(onto_new)).load()
sync_reasoner(onto) 


* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/name belongs to more than one entity types: [owl.AnnotationProperty, owl.DatatypeProperty]; I'm trying to fix it...
* Owlready2 * Running HermiT...
    java -Xmx2000M -cp C:\Users\chern\anaconda3\envs\catnew\lib\site-packages\owlready2\hermit;C:\Users\chern\anaconda3\envs\catnew\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/chern/AppData/Local/Temp/tmpkb_yjyn2
* Owlready2 * HermiT took 109.26907300949097 seconds
* Owlready * Reparenting obo.RO_0002524: {owl.ObjectProperty, obo.RO_0002526, obo.RO_0002522, obo.BFO_0000051} => {obo.BFO_0000051, obo.RO_0002526, obo.RO_0002522}
* Owlready * Reparenting obo.RO_0002012: {owl.ObjectProperty, obo.RO_0002418, obo.BFO_0000050} => {obo.RO_0002418, obo.BFO_0000050}
* Owlready * Reparenting core.narrower: {owl.ObjectProperty, core.narrowerTransitive} => {core.narrowerTransitive}
* Owlready * Reparenting property.AFX_00027

In [92]:
#query function for reaction retrieval
def get_reaction(reac=None,doi=None,include_all=False):
    if doi!=None:
        doi= '"{}".}}'.format(doi)

    if reac==None and doi!=None: 
        sparqlstr="""    
            SELECT ?reacLabel
            WHERE{
                ?reaction rdf:type owl:NamedIndividual.
                ?reaction rdf:type ?type.
                ?type rdfs:subClassOf* obo:BFO_0000015.
                ?reaction rdfs:label ?reacLabel
                ?reaction afo:mentioned_in ?mention.
                ?mention afo:has_doi """ +doi
    elif reac==None and doi==None:
        sparqlstr="""    
            SELECT ?reacLabel ?doi
            WHERE{
                ?reaction rdf:type owl:NamedIndividual.
                ?reaction rdf:type ?type.
                ?type rdfs:subClassOf* obo:BFO_0000015.
                ?reaction rdfs:label ?reacLabel
                ?reaction afo:mentioned_in ?mention.
                ?mention afo:has_doi ?doi.}"""

    elif include_all==True:
        sparqlstr="""
            SELECT ?label ?doi
            WHERE{
                ?reaction rdf:type owl:NamedIndividual.
                ?reaction rdf:type ?type.
                ?type rdfs:subClassOf* obo:BFO_0000015.
                ?reaction rdfs:label ?label.
                FILTER regex(STR(?label),"hydroformylation","i").
                ?reaction afo:mentioned_in ?mention.
                ?mention afo:has_doi ?doi}"""
    else: 
        regex_reac='"{}"'.format(reac)
        sparqlstr="""
            SELECT ?doi
            WHERE{
                ?reaction rdf:type owl:NamedIndividual.
                ?reaction rdf:type ?type.
                ?type rdfs:subClassOf* obo:BFO_0000015.
                ?reaction rdfs:label """ +regex_reac+""".
                ?reaction afo:mentioned_in ?mention.
                ?mention afo:has_doi ?doi}""" 
            #BFO_0000015="process ";reactions are individuals of process subclasses
    

    sparqlstr= """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX obo: <http://purl.obolibrary.org/obo/>
    PREFIX afo: <http://purl.allotrope.org/voc/afo/merged/REC/2023/03/merged-without-qudt-and-inferred#>
    PREFIX new:  <http://www.semanticweb.org/chern/ontologies/2023/10/new_onto.owl#>
    """+sparqlstr        
    try:
        reaction_list= list(default_world.sparql(sparqlstr))
    except:
        reaction_list= []
    return reaction_list   

In [95]:
#queries for reaction entities
doi = r'10.1016/0304-5102(93)87113-m'
list_reac_doi = get_reaction(reac=None,doi=doi) #get list of all reactions and corresponding doi
#print(list_reac_doi)
reac_all = [*set([i[0].lower() for i in list_reac_doi])]
print(reac_all)

['hydroformylation']


In [10]:
#get the list of all publications in ontology where same reactions as in input publication were mentioned
doi = r'10.1016/1381-1169(96)00243-9'
list_reac_doi = get_reaction(reac=None,doi=doi) #get list of all reactions mentioned in given doi (doi needs to be part of ontology)
same_reac_doi = []
for i in list_reac_doi:
    reac_doi = get_reaction(reac = i[0],doi = None)
    for c in reac_doi:
        if c not in same_reac_doi and c[0] != doi:
            same_reac_doi.append(c)#output example: [['10.1016/0304-5102(93)87113-m'],['10.1016/1381-1169(96)00243-9']]
print(list_reac_doi)
print(same_reac_doi)

[['hydroformylation'], ['atmospheric hydroformylation']]
[['10.1016/0304-5102(93)87113-m'], ['10.1021/acscatal.1c02014.s001'], ['10.1021/acscatal.1c04359'], ['10.1021/acscatal.7b00499.s001'], ['10.1021/jacs.1c09665.s001'], ['10.1016/s0920-5861(00)00261-3'], ['10.1016/j.apcata.2013.10.019'], ['10.1016/s1381-1169(97)00035-6'], ['10.1021/acs.iecr.9b03598.s001'], ['10.1021/acsami.0c21749.s001'], ['10.1021/acscatal.0c04684.s001'], ['10.1021/jacs.2c11075.s001']]


In [11]:
#get the list of publications which have "hydroformulation" reaction
list_reac_doi = get_reaction(reac="hydroformylation",doi=None)  #output example: [['10.1016/0304-5102(93)87113-m'],['10.1016/1381-1169(96)00243-9']]
print(list_reac_doi)

[['10.1016/0304-5102(93)87113-m'], ['10.1021/acscatal.1c02014.s001'], ['10.1021/acscatal.1c04359'], ['10.1021/acscatal.7b00499.s001'], ['10.1021/jacs.1c09665.s001'], ['10.1016/1381-1169(96)00243-9'], ['10.1016/s0920-5861(00)00261-3'], ['10.1016/j.apcata.2013.10.019'], ['10.1016/s1381-1169(97)00035-6'], ['10.1021/acs.iecr.9b03598.s001'], ['10.1021/acsami.0c21749.s001'], ['10.1021/acscatal.0c04684.s001'], ['10.1021/jacs.2c11075.s001']]


In [12]:
#get list of all publication which have "hydroformulation" and reactions which have "hydroformylation" in label
list_reac_doi = get_reaction(reac="hydroformylation",doi=None, include_all=True)
print(list_reac_doi)

[['hydroformylation', '10.1016/0304-5102(93)87113-m'], ['hydroformylation', '10.1021/acscatal.1c02014.s001'], ['hydroformylation', '10.1021/acscatal.1c04359'], ['hydroformylation', '10.1021/acscatal.7b00499.s001'], ['hydroformylation', '10.1021/jacs.1c09665.s001'], ['hydroformylation', '10.1016/1381-1169(96)00243-9'], ['hydroformylation', '10.1016/s0920-5861(00)00261-3'], ['hydroformylation', '10.1016/j.apcata.2013.10.019'], ['hydroformylation', '10.1016/s1381-1169(97)00035-6'], ['hydroformylation', '10.1021/acs.iecr.9b03598.s001'], ['hydroformylation', '10.1021/acsami.0c21749.s001'], ['hydroformylation', '10.1021/acscatal.0c04684.s001'], ['hydroformylation', '10.1021/jacs.2c11075.s001'], ['atmospheric hydroformylation', '10.1016/1381-1169(96)00243-9'], ['heterogeneous hydroformylation', '10.1021/acscatal.1c04359'], ['Hydroformylation', '10.1021/jacs.1c09665.s001']]


In [41]:
#query function for catalyst retrieval
def get_catalyst(cat=None,doi=None,include_all=False):
    if doi!=None:
        doi= '"{}".}}'.format(doi) 
    
    if cat ==None:
        sparqlstr="""
    WHERE{
        {
        		?catalyst_e rdf:type owl:NamedIndividual.
		        ?catalyst_e rdf:type ?type.
        		?type rdfs:subClassOf* role:AFRL_0000217.
                ?catalyst_e rdfs:label ?catalyst.
        }
        UNION
    	{
        		?catalyst_full rdf:type owl:NamedIndividual.
        	 	?catalyst_full rdf:type ?type1.
        		?type1 rdfs:subClassOf* ?chem_substance.
                ?chem_substance rdfs:label 'chemical substance'.
                ?catalyst_e afo:catalytic_component_of ?catalyst_full.	
                ?catalyst_e rdfs:label ?catalyst.

                
                
        }
    """
    
        if include_all==True: 
        
            sparqlstr=sparqlstr+"""
            UNION
                 {
                    ?catalyst_e  rdf:type owl:NamedIndividual.
                    ?catalyst_e obo:RO_0000087 ?cat_role.
                    ?cat_role rdfs:label "catalyst role".
                    ?catalyst_e rdfs:label ?catalyst.

                    
                             FILTER NOT EXISTS {

                            ?catalyst_chem rdf:type ?type3.
                            ?type3 rdfs:subClassOf* ?chem_substance.
                            ?chem_substance rdfs:label 'chemical substance'.
                            ?catalyst_e rdf:type ?chem_substance    
                              }      
                     }
                 """

    if doi==None:
        select= """SELECT ?catalyst_e ?catalyst ?doi"""
        sparqlstr=sparqlstr+"""
        	?catalyst_e afo:mentioned_in ?mention.
            ?mention afo:has_doi ?doi}
            """
    else:
        select= """SELECT ?catalyst_e ?catalyst """
        sparqlstr=sparqlstr+"""
        	?catalyst_e afo:mentioned_in ?mention.
            ?mention afo:has_doi """+doi
        
    sparqlstr="""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX obo: <http://purl.obolibrary.org/obo/>
    PREFIX afo: <http://purl.allotrope.org/voc/afo/merged/REC/2023/03/merged-without-qudt-and-inferred#>
    PREFIX role: <http://purl.allotrope.org/ontologies/role#>
    PREFIX new:  <http://www.semanticweb.org/chern/ontologies/2023/10/new_onto.owl#>
    
    """+select+sparqlstr
    print(sparqlstr)
    catalyst_list=list(default_world.sparql(sparqlstr))
    
    return catalyst_list
        #FILTER NOT EXISTS {
        #              ?catalyst_e obo:RO_0000087 ?support_role.
        #              ?support_role rdfs:label "support role"
        #}

In [96]:

def get_synonyms(ent_list,is_cat=False):
    ent_dict={}
    entity_all= [] 
    class_list= []
    for i in range(len(ent_list)):
            super_class=ent_list[i][0].is_a
            ancestors= [i.label for i in super_class[0].ancestors()]
            if  is_cat==True and super_class[0].label=='chemical substance': #super_class[0].label=='catalyst role' or
                continue
            elif is_cat==True and 'catalyst role' in ancestors:
                ent_dict[ent_list[i][1]]=[ent_list[i][1]]
            else:
                if super_class[0] not in class_list:
                    class_list.extend(super_class)

    def_id = ["hasRelatedSynonym", "hasExactSynonym","comment"]
    temp_class_label=[]
    for i in range(len(class_list)):
            temp_class = class_list[i]
            #check, if label and definition are not empty:
            if temp_class.prefLabel:
                    # if preferred label is not empty, use it as class label
                    temp_class_label = temp_class.prefLabel[0]                
            elif temp_class.label:
                        # if label is not empty, use it as class label
                        temp_class_label = temp_class.label[0]
            else:
                temp_class_label=[]

               # except:
                #    temp_class_label = []
                 #   print("Label for class {} not determined!".format(str(temp_class)))


            if temp_class_label and temp_class_label not in ent_dict.keys():
                # if class got a label which is not empty, search for Related and Exact synonyms                    
                ent_dict[temp_class_label] = getattr(temp_class,def_id[0])
                ent_dict[temp_class_label].extend(getattr(temp_class,def_id[1]))
                ent_dict[temp_class_label].extend([i for i in getattr(temp_class,def_id[2]) if i != 'created automatically'])
                ent_dict[temp_class_label]=[*set(ent_dict[temp_class_label])]
                if ent_dict[temp_class_label] and temp_class_label not in ent_dict[temp_class_label]: 
                    ent_dict[temp_class_label].append(temp_class_label)
                elif not ent_dict[temp_class_label]:    
                    ent_dict[temp_class_label] = [temp_class_label]
                  
    for v in ent_dict.values():
        for value in v:
            if value not in entity_all:
                entity_all.append(value)
    return entity_all,ent_dict


    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX obo: <http://purl.obolibrary.org/obo/>
    PREFIX afo: <http://purl.allotrope.org/voc/afo/merged/REC/2023/03/merged-without-qudt-and-inferred#>
    PREFIX role: <http://purl.allotrope.org/ontologies/role#>
    PREFIX new:  <http://www.semanticweb.org/chern/ontologies/2023/10/new_onto.owl#>
    
    SELECT ?catalyst_e ?catalyst 
    WHERE{
        {
        		?catalyst_e rdf:type owl:NamedIndividual.
		        ?catalyst_e rdf:type ?type.
        		?type rdfs:subClassOf* role:AFRL_0000217.
                ?catalyst_e rdfs:label ?catalyst.
        }
        UNION
    	{
        		?catalyst_full rdf:type owl:NamedIndividual.
        	 	?catalyst_full rdf:type ?type1.
        		?type1 rdfs:subClassOf* ?chem_substance.
                ?chem_substance rdfs:label 'c

In [50]:
def get_support(sup=None,doi=None):
    if doi!=None:
        doi= '"{}".}}'.format(doi)
    if sup==None:
        
        sparqlstr="""
        WHERE{
                            ?catalyst_full rdf:type ?type.
                            ?type rdfs:subClassOf* ?chem_substance.
                            ?chem_substance rdfs:label 'chemical substance'.
                            ?catalyst_full rdfs:label ?catalyst.
                            ?support_e afo:support_component_of ?catalyst_full.
                            ?support_e rdfs:label ?support.
                            """
    if doi==None:
        select= """SELECT ?support_e ?support ?catalyst ?doi"""
        sparqlstr=sparqlstr+"""
        	?support_e afo:mentioned_in ?mention.
            ?mention afo:has_doi ?doi}
            """
    else:
        select= """SELECT ?support_e ?support ?catalyst"""
        sparqlstr=sparqlstr+"""
        	?support_e afo:mentioned_in ?mention.
            ?mention afo:has_doi """+doi
    
    sparqlstr="""    
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX obo: <http://purl.obolibrary.org/obo/>
    PREFIX afo: <http://purl.allotrope.org/voc/afo/merged/REC/2023/03/merged-without-qudt-and-inferred#>
    PREFIX role: <http://purl.allotrope.org/ontologies/role#>
    PREFIX new:  <http://www.semanticweb.org/chern/ontologies/2023/10/new_onto.owl#>
    """+select+sparqlstr
    support_list=list(default_world.sparql(sparqlstr))
    print(sparqlstr)
    return support_list

    
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX obo: <http://purl.obolibrary.org/obo/>
    PREFIX afo: <http://purl.allotrope.org/voc/afo/merged/REC/2023/03/merged-without-qudt-and-inferred#>
    PREFIX role: <http://purl.allotrope.org/ontologies/role#>
    PREFIX new:  <http://www.semanticweb.org/chern/ontologies/2023/10/new_onto.owl#>
    SELECT ?support_e ?support ?catalyst
        WHERE{
                            ?catalyst_full rdf:type ?type.
                            ?type rdfs:subClassOf* ?chem_substance.
                            ?chem_substance rdfs:label 'chemical substance'.
                            ?catalyst_full rdfs:label ?catalyst.
                            ?support_e afo:support_component_of ?catalyst_full.
                            ?support_e rdfs:label ?support.
        

In [66]:
def get_entList(list_type,entity= None,doi=None): #list_type one of ['reactant','product']
    if doi!=None:
        doi= '"{}".}}'.format(doi)
    if entity== None:
        sparqlstr='''
        WHERE{
                ?ent_e rdf:type owl:NamedIndividual.
                ?entRole rdf:type owl:NamedIndividual.
                ?entRole rdfs:label "''' +list_type+''' role".
                ?ent_e obo:RO_0000087 ?entRole.
                ?ent_e rdfs:label ?ent.
            '''
    if doi==None:
        select= '''SELECT ?ent_e ?ent ?doi'''
        sparqlstr= sparqlstr+'''?ent_e afo:mentioned_in ?mention.
            ?mention afo:has_doi ?doi}'''
            
    else:
        select=''' SELECT ?ent_e ?ent''' 
        sparqlstr= sparqlstr + '''?ent_e afo:mentioned_in ?mention.
            ?mention afo:has_doi''' +doi
    sparqlstr="""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX obo: <http://purl.obolibrary.org/obo/>
    PREFIX afo: <http://purl.allotrope.org/voc/afo/merged/REC/2023/03/merged-without-qudt-and-inferred#>
    PREFIX new:  <http://www.semanticweb.org/chern/ontologies/2023/10/new_onto.owl#>
    """+select+sparqlstr
    print(sparqlstr)
    ent_list=list(default_world.sparql(sparqlstr))
    return ent_list

In [98]:
sup_list=get_support(sup=None,doi='10.1016/0304-5102(93)87113-m')
_,sup_all=get_synonyms(support_list,is_cat=False)
cat_list=get_catalyst(cat=None,doi='10.1016/0304-5102(93)87113-m',include_all=True)
_,cat_all=get_synonyms(cat_list,is_cat=True)
reactant_list=get_entList('reactant',entity= None,doi='10.1016/0304-5102(93)87113-m')
reactant_all,_=get_synonyms(reactant_list,is_cat=False)
product_list=get_entList('product',entity= None,doi='10.1016/0304-5102(93)87113-m')
product_all,_=get_synonyms(product_list,is_cat=False)


    
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX obo: <http://purl.obolibrary.org/obo/>
    PREFIX afo: <http://purl.allotrope.org/voc/afo/merged/REC/2023/03/merged-without-qudt-and-inferred#>
    PREFIX role: <http://purl.allotrope.org/ontologies/role#>
    PREFIX new:  <http://www.semanticweb.org/chern/ontologies/2023/10/new_onto.owl#>
    SELECT ?support_e ?support ?catalyst
        WHERE{
                            ?catalyst_full rdf:type ?type.
                            ?type rdfs:subClassOf* ?chem_substance.
                            ?chem_substance rdfs:label 'chemical substance'.
                            ?catalyst_full rdfs:label ?catalyst.
                            ?support_e afo:support_component_of ?catalyst_full.
                            ?support_e rdfs:label ?support.
        

In [99]:
product_all

[]

In [111]:

#reac_list=[['hydroformylation'], ['atmospheric hydroformylation']]
#cat_list=[['Co'],['Rh']]
#sup_list=[['SiO2']]

columns= ['eid',
          'doi',
          'pii',
          'pubmed_id',
          'title',
          'subtype',
          'subtypeDescription',
          'creator',
          'afid',
          'affilname',
          'affiliation_city',
          'affiliation_country',
          'author_count',
          'author_names',
          'author_ids',
          'author_afids',
          'coverDate',
          'coverDisplayDate',
          'publicationName',
          'issn',
          'source_id',
          'eIssn',
          'aggregationType',
          'volume',
          'issueIdentifier',
          'article_number',
          'pageRange',
          'description',
          'authkeywords',
          'citedby_count',
          'openaccess',
          'freetoread',
          'freetoreadLabel',
          'fund_acr',
          'fund_no',
          'fund_sponsor',
          'query']
df_all = pd.DataFrame(columns=columns)
results=[]
if not reac_all:
    reac_all.append(' ')
if not reactant_all:
    reactant_all.append(' ')
if not product_all:
    product_all.append(' ')

for r in reac_all:
        for react in reactant_all:
            for prod in product_all:
                for k_cat,v_cat in cat_all.items():
                    for k_sup,v_sup in sup_all.items():
                        if k_sup == k_cat:
                            continue
                        else:
                            for cat in v_cat:
                                for sup in v_sup:
                                    query = 'TITLE-ABS-KEY("{}"AND"{}"AND"{}"AND"{}"AND"{}")'.format(r,cat,sup,react,prod)
                                    print(query)
                                    result = ScopusSearch(query, view='STANDARD',verbose=True, subscriber=True)
                                    results.append(result)
                                    df = pd.DataFrame(pd.DataFrame(result.results))
                                    df['query'] = [query for _ in range(result.get_results_size())]
                                    df_all = pd.concat([df_all, df], axis=0, ignore_index=True)
    

distinct_df=df_all.drop_duplicates(['eid']).reset_index(drop=True)

distinct_df['coverDate']=pd.to_datetime(distinct_df['coverDate'])

TITLE-ABS-KEY("hydroformylation"AND"Rh"AND"aluminio"AND"Propene"AND" ")


1it [00:00, ?it/s]

TITLE-ABS-KEY("hydroformylation"AND"Rh"AND"Al"AND"Propene"AND" ")


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]

TITLE-ABS-KEY("hydroformylation"AND"Rh"AND"13Al"AND"Propene"AND" ")


...

1it [00:00, ?it/s]

TITLE-ABS-KEY("hydroformylation"AND"trichlororhodium"AND"H(-)"AND"methylethene"AND" ")


1it [00:00, ?it/s]

TITLE-ABS-KEY("hydroformylation"AND"trichlororhodium"AND"hydride"AND"methylethene"AND" ")


1it [00:00, ?it/s]


In [104]:
reactant_all

['Propene',
 'propene',
 'methylethylene',
 'propylene',
 'prop-1-ene',
 'R-1270',
 '1-propylene',
 'CH2=CH-CH3',
 '1-propene',
 'methylethene']

In [112]:
#filter found publication according the date
start_date = '2019-01-01' #or None
end_date = '2023-12-31' 
if start_date!= None:
    filtered_df = distinct_df[(distinct_df['coverDate'] >= start_date) & (distinct_df['coverDate'] <= end_date)]
else:
    filtered_df = distinct_df

onto_doi_list=list(default_world.sparql('''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX afo: <http://purl.allotrope.org/voc/afo/merged/REC/2023/03/merged-without-qudt-and-inferred#>
PREFIX new:  <http://www.semanticweb.org/chern/ontologies/2023/10/new_onto.owl#>
SELECT ?doi
WHERE{
          ?publication rdf:type owl:NamedIndividual.
          ?publication rdf:type ?type.
          ?type rdfs:subClassOf* afo:publication.
          ?publication afo:has_doi ?doi
        }'''))
    
idx_none=[]
for row in filtered_df.itertuples():
    if row.doi == None:
        idx_none.append(row.Index)
filtered_df_doi= filtered_df.drop(index=idx_none)

query_doi_list=list(filtered_df['doi'])
idx_copy=[]
for d in onto_doi_list:
    if d[0] in query_doi_list:
        idx_copy.extend(list(filtered_df_doi.query('doi == "{}"'.format(d[0])).index))
filtered_df_doi_new = filtered_df.drop(index=idx_copy)


In [113]:
#save from scopus quieried publication information in excel
with pd.ExcelWriter('./output/output.xlsx', engine='xlsxwriter') as writer: #install xlsxwriter with pip
    # Write the first dataframe to a sheet named 'all'
    df_all.to_excel(writer, sheet_name='all', index=False)

    # Write the second dataframe to a sheet named 'distinct_eid'
    distinct_df.to_excel(writer, sheet_name='distinct_eid', index=False)
    
    #Write the third dataframe to a sheet named 'filtered_distinct'
    filtered_df.to_excel(writer, sheet_name='filtered_distinct', index=False)
    
    #Write the fourth dataframe to a sheet named 'filtered_with doi'
    filtered_df_doi.to_excel(writer, sheet_name='filtered_with_doi', index=False)
    
    #Write the fourth dataframe to a sheet named 'filtered_new_doi'
    filtered_df_doi_new.to_excel(writer, sheet_name='filtered_new_doi', index=False)